In [15]:
from __future__ import print_function, division
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import Dataset
from skimage import io, transform
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import pandas as pd
import numpy as np
import torch
import os

In [2]:
def add_delta(df): 
    """
    adds column delta_w_z_right to df data
    """
    delta_w_z_right_r_unit_v_col = []

    for row in range(len(df)-1):    
        curr_type = df.at[row, "types_x"]
        next_type = df.at[row+1, "types_x"]

        if curr_type == next_type: 
            curr_w_z_right = df.at[row, "w_z_right_r_unit_v"]
            next_w_z_right = df.at[row+1, "w_z_right_r_unit_v"]
            delta = next_w_z_right - curr_w_z_right
        else: #CHANGE
            curr_w_z_right = df.at[row, "w_z_right_r_unit_v"]
            prev_w_z_right = df.at[row-1, "w_z_right_r_unit_v"]
            delta = curr_w_z_right - prev_w_z_right
            
        delta_w_z_right_r_unit_v_col += [delta]

    last_w_z_right = df_video.at[len(df)-1, "w_z_right_r_unit_v"]
    prev_w_z_right = df_video.at[len(df)-2, "w_z_right_r_unit_v"]
    delta = curr_w_z_right - prev_w_z_right
    delta_w_z_right_r_unit_v_col += [delta]    

    delta_w_z_right_r_unit_v_col = np.array(delta_w_z_right_r_unit_v_col).reshape(-1, 1)

    df["delta_w_z_right"] = delta_w_z_right_r_unit_v_col * 10

In [3]:
remove_cols = [
       'Unnamed: 0', 'index', 'e_x_right', 'e_y_right', 'e_z_right',
       'w_x_right', 'w_y_right', 'w_z_right', 's_x_right', 's_y_right',
       's_z_right', 'c_x_right', 'c_y_right', 'c_z_right', 's_x_left',
       's_y_left', 's_z_left', 'e_x_left', 'e_y_left', 'e_z_left', 'w_x_left',
       'w_y_left', 'w_z_left', 'types_y', 'v_types', "trial",
              
#        'e_x_right_r', 'e_y_right_r', 'e_z_right_r', 'w_x_right_r',
#        'w_y_right_r', 'w_z_right_r', 's_x_right_r', 's_y_right_r',
#        's_z_right_r', 'c_x_right_r', 'c_y_right_r', 'c_z_right_r',
#        's_x_left_r', 's_y_left_r', 's_z_left_r', 'e_x_left_r', 'e_y_left_r',
#        'e_z_left_r', 'w_x_left_r', 'w_y_left_r', 'w_z_left_r',
]

df_video = pd.read_csv("./data/video_all_data.csv")
df_video = df_video.drop(columns=remove_cols)
add_delta(df_video)

print(len(df_video))

X_train, X_test = train_test_split(df_video,test_size=0.8)

print(len(X_train) + len(X_test))

X_train.to_csv(r'./data/test2_video.csv')
X_test.to_csv(r'./data/train2_video.csv')

print(df_video.keys())

15726
15726
Index(['types_x', 'e_x_right_r', 'e_y_right_r', 'e_z_right_r', 'w_x_right_r',
       'w_y_right_r', 'w_z_right_r', 's_x_right_r', 's_y_right_r',
       's_z_right_r', 'c_x_right_r', 'c_y_right_r', 'c_z_right_r',
       's_x_left_r', 's_y_left_r', 's_z_left_r', 'e_x_left_r', 'e_y_left_r',
       'e_z_left_r', 'w_x_left_r', 'w_y_left_r', 'w_z_left_r',
       'e_x_right_r_unit_v', 'e_y_right_r_unit_v', 'e_z_right_r_unit_v',
       'w_x_right_r_unit_v', 'w_y_right_r_unit_v', 'w_z_right_r_unit_v',
       'delta_w_z_right'],
      dtype='object')


In [13]:
df_sensor = pd.read_csv("./data/sensor_all_data.csv")
print(df_sensor.keys())

Index(['Unnamed: 0', 'e_x_right', 'e_y_right', 'e_z_right', 'index',
       'move_type', 'p_num', 's_types', 's_x_right', 's_y_right', 's_z_right',
       'time', 'trial', 'trial_extra', 'types', 'w_x_right', 'w_y_right',
       'w_z_right', 'e_x_right_r_unit_v', 'e_y_right_r_unit_v',
       'e_z_right_r_unit_v', 'w_x_right_r_unit_v', 'w_y_right_r_unit_v',
       'w_z_right_r_unit_v'],
      dtype='object')


In [4]:
class FramesDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.frames = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.frames)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        data = self.frames.iloc[idx, :]    
        exc_type = int(data['types_x'][-2:])-1   # NOTICE: exc01 is at index 0
        data = torch.tensor(data[2:])
                
        if self.transform:
            data = self.transform(data)
        return data, exc_type

In [5]:
train = FramesDataset(csv_file = 'data/train2_video.csv', root_dir='data/')
test = FramesDataset(csv_file = 'data/test2_video.csv', root_dir='data/')

trainset = torch.utils.data.DataLoader(train, batch_size=5, shuffle=True)
testset = torch.utils.data.DataLoader(test, batch_size=5, shuffle=True)

In [6]:
# for i, data in enumerate(trainset, 0):
#     x, y = data
#     print(data)

In [7]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()  
        self.fc1 = nn.Linear(28, 120) #no 2 with 64
#         self.fc2 = nn.Linear(120, 64)
        self.fc3 = nn.Linear(120, 15)
    
        # ADD weights for delta HERE
        # nn.Parameter
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)   # taking log_softmax not needed? for optimization?

net = Net()

In [8]:
# algorithm for first-order gradient-based optimization, built into PyTorch.
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [9]:
EPOCHS = 8

for epoch in range(EPOCHS):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainset, 0):
        # get the inputs; data is dictionary
        X, y = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        output = net(X)
        loss = F.nll_loss(output, y)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 1000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

print('Finished Training')

[1,  1000] loss: 1.347
[1,  2000] loss: 0.752
[2,  1000] loss: 0.484
[2,  2000] loss: 0.399
[3,  1000] loss: 0.295
[3,  2000] loss: 0.280
[4,  1000] loss: 0.223
[4,  2000] loss: 0.205
[5,  1000] loss: 0.178
[5,  2000] loss: 0.181
[6,  1000] loss: 0.162
[6,  2000] loss: 0.147
[7,  1000] loss: 0.145
[7,  2000] loss: 0.128
[8,  1000] loss: 0.111
[8,  2000] loss: 0.125
Finished Training


In [10]:
correct = 0
total = 0
with torch.no_grad():
    for data in testset:
        X, y = data
        output = net(X)
        for index, i in enumerate(output):
            if torch.argmax(i) == y[index]:
                correct += 1
            total += 1

print("Accuracy: ", round(correct/total, 3))

Accuracy:  0.961


EPOCHS = 3
<br>
(28,120) -> (120,15)<br>
0.926, 0.914, 0.92 <br>
vs. 0.928, 0.906, 0.91<br>
vs. 0.893 <br>
vs. .911<br>

delta * 100<br>
vs. delta * 10 <br>
vs. delta <br>
vs. no delta<br>
<br>
<br>
EPOCHS = 8<br>
(28,120) -> (120,15)<br>
0.952, 0.961<br>
vs. 0.936, 0.943<br>

delta * 10<br>
vs. no delta

In [ ]:
# ADD Make seed_pytorch